In [1]:
import sys
import torch
from torch import nn

/home/cstainsby/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Pull in my modules

In [2]:
# import my modules 
from path import get_model_folder_path
sys.path.append(get_model_folder_path())

from basicModel import Transformer

from data_utils import Batch, data_gen
from model_utils import fit_model

562 batches of size 16
187 batches of size 16


### Basic Model creation

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = Transformer(
    num_tokens=4, dim_model=8, num_heads=2, num_encoder_layers=3, num_decoder_layers=3, dropout_p=0.1
).to(device)
opt = torch.optim.SGD(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

### Create Random Data

In [4]:
train_data = generate_random_data(9000)
test_data = generate_random_data(3000)

NameError: name 'generate_random_data' is not defined

#### Batchify the data

In [ ]:
Batch()

### Training

In [ ]:
fit_model(model, opt, loss_fn, 5)

EPOCH: 0


TypeError: run_epoch() got an unexpected keyword argument 'optimizer'